In [10]:
!pip install nltk textblob
!pip install torch transformers
!pip install happytransformer

In [11]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.metrics import f1_score, confusion_matrix
import string
import textwrap
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoModelForCausalLM, AutoTokenizer, pipeline, AdamW

import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader, TensorDataset, random_split


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### EDA and Cleaning

In [13]:
#df = pd.read_csv('https://drive.google.com/file/d/10p5JAvgPD-AQNUzXSJ17t3bfEor6Pm5h/view?usp=sharing',nrows=20)
df = pd.read_csv('/content/final_dataset_task3.csv',nrows=20)
df1_lower = df.copy()
df1_lower['Job_description'] = df['Job_description'].str.lower()

df1_lower.head()

,Unnamed: 0,Job_description,Length,Lemmatized_description,male_bias,female_bias,racial_bias,% male_bias,% female_bias,% racial_bias,male_bias_yes,female_bias_yes,racial_bias_yes,salary,numerical_salary,gender_bias,ratio_female_male_bias
0,0,smb account executive identify qualify new opp...,303,"['smb', 'account', 'executive', 'identify', 'q...",6,2,0,1.980198,0.660066,0.000000,1,1,0,NaN,0.0,Male,0.333333
1,1,looking key new member join small talented pro...,246,"['looking', 'key', 'new', 'member', 'join', 's...",6,2,0,2.439024,0.813008,0.000000,1,1,0,NaN,0.0,Male,0.333333
2,2,looking data analyst software developer work r...,144,"['looking', 'data', 'analyst', 'software', 'de...",0,1,1,0.000000,0.694444,0.694444,0,1,1,NaN,0.0,Female,0.000000
3,3,creative virtual therefore seeking energetic e...,149,"['creative', 'virtual', 'therefore', 'seeking'...",1,3,0,0.671141,2.013423,0.000000,1,1,0,NaN,0.0,Female,3.000000
4,4,essential skills minimum bachelor degree equiv...,94,"['essential', 'skill', 'minimum', 'bachelor', ...",0,0,0,0.000000,0.000000,0.000000,0,0,0,NaN,0.0,Female,0.000000


In [14]:
duplicates_per_column = df1_lower.apply(lambda x: x.duplicated().sum())
print(duplicates_per_column)

Unnamed: 0                 0
Job_description            0
Length                     0
Lemmatized_description     0
male_bias                 13
female_bias               13
racial_bias               18
% male_bias                5
% female_bias              4
% racial_bias             18
male_bias_yes             18
female_bias_yes           18
racial_bias_yes           18
salary                    19
numerical_salary          19
gender_bias               18
ratio_female_male_bias     9
dtype: int64


In [15]:
df1_lower = df1_lower.drop_duplicates( subset =['Job_description'], keep ='first')

In [16]:
df1_lower.shape

(20, 17)

In [17]:
df1_lower = df1_lower[['Job_description']].copy()
df1_lower.head()

,Job_description
0,smb account executive identify qualify new opp...
1,looking key new member join small talented pro...
2,looking data analyst software developer work r...
3,creative virtual therefore seeking energetic e...
4,essential skills minimum bachelor degree equiv...


In [18]:
def remove_html_tags(text):
    if isinstance(text, str):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)
    else:
        return text

columns_to_clean = ['Job_description']

for column in columns_to_clean:
    df[column] = df[column].apply(remove_html_tags)


In [19]:
def preprocess(q):

    q=str(q).lower().strip()
    q = q.replace("â€™","'") # added by Saurabh
    q = q.replace("job description: ","")
    q = q.replace("job description ","")
    q = q.replace("job description","")
    text = '''
    Job Details


    Job Location
    Saudi Arabia


    Company Industry
    Other Business Support Services


    Company Type
    Unspecified


    Employment Type
    Unspecified


    Monthly Salary Range
    Unspecified


    Number of Vacancies
    Unspecified
    '''
    q = q.replace(text,"")

    q = q.replace("Â","")
    q = q.replace("Ã","")

    q = q.replace("$","dollar")
    q = q.replace("₹","rupee")
    q = q.replace("%","percent")
    q = q.replace("@","at")
    q = q.replace("€","euro")

    q = q.replace('[math]', '')

    q = q.replace(', 000,000,000', 'b ')
    q = q.replace(', 000,000', 'm ')
    q = q.replace(', 000', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)


    #decontracting words
    contractions = {
    "ain't": "am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]
        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ill", " will")

    #removing html tags
    q = BeautifulSoup(q)
    q = q.get_text()

    #removing punctuations
    pattern = re.compile('\W')

    q=re.sub(pattern , ' ', q).strip()

    return q

# preprocess
df1_lower['Job_description'] = df1_lower['Job_description'].apply(preprocess)
# remove extra spaces
df1_lower['Job_description'] = df1_lower['Job_description'].str.strip().str.replace(r'\s+', ' ', regex=True)

In [20]:
masculine_words = ['ambition', 'driven', 'lead', 'persist', 'principle', 'decision', 'superior', 'individual', 'assertive', 'strong', 'hierarchical', 'rigid', 'silicon valley', 'stock options', 'takes risk', 'workforce', 'autonomous', 'ping pong', 'pool table', 'must', 'competitive', 'he', 'his', 'himself', 'confident', 'active', 'aggressive', 'ambitious', 'fearless', 'headstrong', 'defensive', 'independent', 'dominant', 'outspoken', 'leader', 'fast paced', 'adventurous', 'analytical', 'decisive', 'determined', 'ninja', 'objective', 'rock star', 'boast', 'challenging', 'courage']
feminine_words = ['thoughtful', 'creative', 'adaptable', 'choose', 'curious', 'excellent', 'flexible', 'multitasking', 'health', 'imaginative', 'intuitive', 'leans in', 'plans for the future', 'resilient', 'self-aware', 'socially responsible', 'trustworthy', 'shup-to-date', 'wellness program', 'nurture', 'teach', 'dependable', 'community', 'serving', 'loyal', 'enthusiasm', 'interpersonal', 'connect', 'commit', 'she', 'agree', 'empathy', 'sensitive', 'affectionate', 'feel', 'support', 'collaborate', 'honest', 'trust', 'understand', 'comapssion', 'share', 'polite', 'kind', 'caring', 'her', 'hers', 'herself', 'feminine', 'cheer', 'communal', 'emotional', 'flatterable', 'gentle', 'interdependent', 'kinship', 'modesty', 'pleasant', 'polite', 'quiet', 'sympathy', 'warm', 'dominant', 'yield']
racial_words = ['native english speaker', 'professionally groomed hair', 'native', 'culture fit', 'non-white', 'clean-shaven', 'neat hairstyle', 'master', 'slave', 'a cakewalk', 'brownbag session', 'spirit animal', 'digital native', 'servant leadership', 'tribe', 'oriental', 'spic', 'english fluency', 'level native', 'illegals', 'eskimo', 'latino', 'latina', 'migrant', 'blacklist', 'whitelist']


In [21]:
df_final = df1_lower.copy()
df_final.head(2)


,Job_description
0,smb account executive identify qualify new opp...
1,looking key new member join small talented pro...


In [22]:
df_final.shape

(20, 1)

In [23]:
df_final = df_final.dropna(subset=['Job_description'])
df_final.shape

(20, 1)

In [24]:
job_descriptions = df_final['Job_description']

def analyze_job_description(job_description):
    tokens = word_tokenize(job_description.lower())

    # Analyze gender (example: finding gendered pronouns)
    gendered_pronouns = [
          'ambition', 'driven', 'lead', 'persist', 'principle', 'decision', 'superior', 'individual', 'assertive', 'strong', 'hierarchical', 'rigid', 'silicon valley', 'stock options', 'takes risk', 'workforce', 'autonomous', 'ping pong', 'pool table', 'must', 'competitive', 'he', 'his', 'himself', 'confident', 'active', 'aggressive', 'ambitious', 'fearless', 'headstrong', 'defensive', 'independent', 'dominant', 'outspoken', 'leader', 'fast paced', 'adventurous', 'analytical', 'decisive', 'determined', 'ninja', 'objective', 'rock star', 'boast', 'challenging', 'courage',
          'thoughtful', 'creative', 'adaptable', 'choose', 'curious', 'excellent', 'flexible', 'multitasking', 'health', 'imaginative', 'intuitive', 'leans in', 'plans for the future', 'resilient', 'self-aware', 'socially responsible', 'trustworthy', 'shup-to-date', 'wellness program', 'nurture', 'teach', 'dependable', 'community', 'serving', 'loyal', 'enthusiasm', 'interpersonal', 'connect', 'commit', 'she', 'agree', 'empathy', 'sensitive', 'affectionate', 'feel', 'support', 'collaborate', 'honest', 'trust', 'understand', 'compassion', 'share', 'polite', 'kind', 'caring', 'her', 'hers', 'herself', 'feminine', 'cheer', 'communal', 'emotional', 'flatterable', 'gentle', 'interdependent', 'kinship', 'modesty', 'pleasant', 'polite', 'quiet', 'sympathy', 'warm', 'dominant', 'yield',
          'native english speaker', 'professionally groomed hair', 'native', 'culture fit', 'non-white', 'clean-shaven', 'neat hairstyle', 'master', 'slave', 'a cakewalk', 'brownbag session', 'spirit animal', 'digital native', 'servant leadership', 'tribe', 'oriental', 'spic', 'english fluency', 'level native', 'illegals', 'eskimo', 'latino', 'latina', 'migrant', 'blacklist', 'whitelist'
      ]
    gendered_words = [word for word in tokens if word in gendered_pronouns]
    gendered_word_count = len(gendered_words)
    if gendered_word_count > 0:
      label = 1
    else:
      label = 0

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    word_freq = FreqDist(filtered_tokens)
    blob = TextBlob(job_description)
    sentiment = blob.sentiment
    sentences = sent_tokenize(job_description)
    sentence_count = len(sentences)

    # Analyze tone (positive 1, negative 0, neutral 0.5)
    if sentiment.polarity > 0:
        tone = 1
    elif sentiment.polarity < 0:
        tone = 0
    else:
        tone = 0.5

    '''
    return {
        "Number of Sentences": sentence_count,
        "Sentiment": tone,
        "Polarity": sentiment.polarity,
        "Subjectivity": sentiment.subjectivity,
        "Word Frequency": word_freq.most_common(10)
    }
    '''
    features = [0.0]*6

    features[0] = sentence_count
    features[1] = tone
    features[2] = sentiment.polarity
    features[3] = sentiment.subjectivity
    features[4] = gendered_word_count
    features[5] = label
    return features


analysis_results = df_final['Job_description'].apply(analyze_job_description)

df_final['sentence_count'] = list(map(lambda x: x[0], analysis_results))
df_final['tone'] = list(map(lambda x: x[1], analysis_results))
df_final['polarity'] = list(map(lambda x: x[2], analysis_results))
df_final['subjectivity'] = list(map(lambda x: x[3], analysis_results))
df_final['gendered_word_count'] = list(map(lambda x: x[4], analysis_results))
df_final['label'] = list(map(lambda x: x[5], analysis_results))

### Model Creation

In [25]:
loss_fn = BCEWithLogitsLoss()
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
texts = df_final['Job_description'].tolist()
labels = df_final['label'].tolist()
max_length = max([len(word_tokenize(text)) for text in texts])
print(max_length)

368


In [27]:
input_data = tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

In [28]:
# Convert the list of labels to a tensor
label_tensor = torch.tensor(labels, dtype=torch.long)

# Combine the training inputs into a TensorDataset
#dataset = TensorDataset(input_data["input_ids"], input_data["attention_mask"], input_data["token_type_ids"], label_tensor)
dataset = TensorDataset(input_data["input_ids"], input_data["attention_mask"], label_tensor)


In [29]:
# Define the sizes
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [30]:
# Create DataLoaders for training and validation
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [31]:
len(train_dataloader), len(validation_dataloader)

(4, 1)

In [32]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=1)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [35]:
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = BCEWithLogitsLoss()

best_val_loss = float("inf")
patience_counter = 0
patience_limit = 1  # Adjust based on your requirements

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        # Load batch to GPU
        #b_input_ids, b_attention_mask, b_token_type_ids, b_labels = batch
        b_input_ids, b_attention_mask, b_labels = batch
        b_input_ids = b_input_ids.to(device)
        b_attention_mask = b_attention_mask.to(device)
        #b_token_type_ids = b_token_type_ids.to(device)
        b_labels = b_labels.to(device)

        # Zero out any previously calculated gradients
        model.zero_grad()

        # Perform a forward pass
        #outputs = model(b_input_ids, attention_mask=b_attention_mask, token_type_ids=b_token_type_ids)
        outputs = model(b_input_ids, attention_mask=b_attention_mask)

        # Get the logits from the model
        logits = outputs.logits.squeeze()

        # Compute loss and accumulate the loss values
        loss = loss_fn(logits, b_labels.float())
        total_loss += loss.item()

        # Perform a backward pass to calculate gradients
        loss.backward()

        # Update parameters
        optimizer.step()

    # Average loss over the training epoch
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch: {epoch + 1}, Train Loss: {avg_train_loss:.4f}")

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0

    # Disable gradient updates
    with torch.no_grad():
        for batch in validation_dataloader:
            b_input_ids, b_attention_mask, b_token_type_ids, b_labels = batch
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_labels = b_labels.squeeze().to(device)#b_labels.to(device)

            #print("b_labels",b_labels)

            #outputs = model(b_input_ids, attention_mask=b_attention_mask, token_type_ids=b_token_type_ids)
            outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask)
            logits = outputs.logits.squeeze()
            #print("logits",logits)

            loss = loss_fn(logits, b_labels.float())
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(validation_dataloader)
    print(f"Epoch: {epoch + 1}, Validation Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience_limit:
        print("Early stopping triggered!")
        break


Epoch: 1, Train Loss: 0.2607


ValueError: ignored

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [37]:
# Directory where model will be saved
save_directory = "/content/drive/My Drive/datasets/JobBiasDataset/roberta_model"

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

('/content/drive/My Drive/datasets/JobBiasDataset/roberta_model/tokenizer_config.json',
 '/content/drive/My Drive/datasets/JobBiasDataset/roberta_model/special_tokens_map.json',
 '/content/drive/My Drive/datasets/JobBiasDataset/roberta_model/vocab.json',
 '/content/drive/My Drive/datasets/JobBiasDataset/roberta_model/merges.txt',
 '/content/drive/My Drive/datasets/JobBiasDataset/roberta_model/added_tokens.json')

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained(save_directory)
model.to(device)
tokenizer = RobertaTokenizer.from_pretrained(save_directory)

In [39]:
model.to(device)

# Step 1: Predictions and True Labels
all_predictions = []
all_true_labels = []
total_eval_accuracy = 0

model.eval()
for batch in test_dataloader:
    #b_input_ids, b_attention_mask, b_token_type_ids, b_labels = batch
    b_input_ids, b_attention_mask, b_token_type_ids, b_labels = batch
    b_input_ids = b_input_ids.to(device)
    b_attention_mask = b_attention_mask.to(device)
    #b_token_type_ids = b_token_type_ids.to(device)
    b_labels = b_labels.to(device)

    with torch.no_grad():
        #outputs = model(b_input_ids, attention_mask=b_attention_mask, token_type_ids=b_token_type_ids)
        outputs = model(b_input_ids, attention_mask=b_attention_mask)

    logits = outputs.logits.squeeze()

    predictions = (logits > 0).int()
    if predictions.dim() == 0:
      all_predictions.extend(predictions)
      all_true_labels.extend(b_labels)
    else:
      all_predictions.extend(predictions.cpu().numpy())
      all_true_labels.extend(b_labels.cpu().numpy())

    # Calculate the accuracy rate
    total_eval_accuracy += (predictions == b_labels).sum().item()
    #break
# Average accuracy over all batches
avg_val_accuracy = total_eval_accuracy / len(validation_dataloader.dataset)


# Step 2: Compute F1-Score
f1 = f1_score(all_true_labels, all_predictions)

# Step 3: Compute Confusion Matrix
conf_matrix = confusion_matrix(all_true_labels, all_predictions)

print(f"Validation Accuracy: {avg_val_accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

ValueError: ignored

### Testing

In [40]:
# Load the trained model and tokenizer
save_directory = "/content/drive/My Drive/datasets/JobBiasDataset/roberta_model"
model = RobertaForSequenceClassification.from_pretrained(save_directory)
tokenizer = RobertaTokenizer.from_pretrained(save_directory)

# Put the model in evaluation mode
model.eval()

# Input text
input_text = "Our client, located in Houston, is actively seeking an experienced Commissioning Machinery Assistant that possesses strong supervisory skills and has an attention to detail. A strong dedication to safety is a must. The ideal candidate will execute all activities while complying with quality requirements and health, environmental, and safety regulations.	Implement pre-commissioning and commissioning procedures for rotary equipment. Execute all activities with subcontractor€™s assigned crew that pertains to the discipline. Ensure effective utilization of commissioning manpower and consumables. Ensure the execution of vendor specialists' field activities with the assigned resources from the sub-contractor per vendor€™s representative plans. Carry out equipment inspections with client representatives and ensure proper certification is produced. Prepare forms for all pending tests and submit signed certificates for final hand over to the certification engineer for QA and QC. Coordinate in the field with vendor representatives. Keep records of all activities. Ensure that safety practices are strictly followed during the execution of activities. Report progress and constraints to the mechanical supervisor. Possible authorization by site manager to receive or issue a Permit To Work according to project Permit To Work procedures. Assist supervisor to expedite pending punch-list items in accordance with the commissioning manager€™s priorities. Assist supervisor to coordinate and supervise construction-support activities during pre-commissioning and commissioning activities. Company Overview: Our client is a premiere engineering, construction, and procurement company that executes large-scale projects internationally."

# Tokenize the input text
inputs = tokenizer(input_text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

# Make the prediction
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
predicted_label = (logits > 0).int().item()

# Convert 0 or 1 label back to a meaningful label if needed
label_mapping = {0: "Negative", 1: "Positive"}
predicted_label_text = label_mapping[predicted_label]

print(f"Predicted Label: {predicted_label_text}")


Predicted Label: Positive


### Generate new BIASED Free Description

In [41]:
from happytransformer import HappyTextToText, TTSettings

In [42]:
import re

def replace_gendered_pronouns(text):
    # Define a dictionary of gendered pronouns and their gender-neutral replacements
    gendered_pronouns = {
        "he": "they",
        "him": "them",
        "his": "their",
        "she": "they",
        "her": "them",
        "hers": "theirs",
    }

    # Use regular expressions to find and replace gendered pronouns in the text
    for pronoun, replacement in gendered_pronouns.items():
        # Use word boundaries to match whole words only
        pattern = r'\b' + re.escape(pronoun) + r'\b'
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

    return text

# Example usage:
original_sentence = "He is a diligent worker, and his dedication is commendable."
rewritten_sentence = replace_gendered_pronouns(original_sentence)
print(rewritten_sentence)



happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
args = TTSettings(num_beams=5, min_length=1)
# Add the prefix "grammar: " before each input
result = happy_tt.generate_text("grammar: "+rewritten_sentence, args=args)
print(result.text) # This sentence has bad grammar.

they is a diligent worker, and their dedication is commendable.


They are a diligent worker, and their dedication is commendable.
